# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups?
            * If so, how do you plan to account for this in your experimental design?

In [ ]:
#Your code here
## Import the packages
import numpy as np
import pandas as pd
from scipy import stats
import math

# For visualizing distributions - optional
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [ ]:
df.nunique()

timestamp    8188
id           6328
group           2
action          2
dtype: int64

In [ ]:
df.dtypes

timestamp    object
id            int64
group        object
action       object
dtype: object

In [ ]:
user_actions = df.groupby("id")["action"].unique().reset_index()
user_actions

,id,action
0,182988,[view]
1,182994,"[view, click]"
2,183089,[view]
3,183136,[view]
4,183141,"[view, click]"
...,...,...
6323,937003,[view]
6324,937073,[view]
6325,937108,[view]
6326,937139,"[view, click]"


In [ ]:
# Users with both "view" and "click"
both_view_and_click = user_actions[user_actions["action"].apply(lambda x: set(["view","click"]).issubset(x))]

print("Number of users who both viewed and clicked:", len(both_view_and_click))

Number of users who both viewed and clicked: 1860


In [ ]:
# Users who only clicked but never viewed
only_clickers = user_actions[user_actions["action"].apply(lambda x: x.tolist() == ["click"])]

print("Users who clicked without viewing:", len(only_clickers))

Users who clicked without viewing: 0


In [ ]:
# Find group assignment per user
user_groups = df.groupby("id")["group"].nunique().reset_index()

# Overlap = users with >1 group
overlap_users = user_groups[user_groups["group"] > 1]

print("Users in both control & experiment:", len(overlap_users))

Users in both control & experiment: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [ ]:
#Your code here
'''
Since the test is categorical, we'll do two proportion z-test to check whether the experimental homepage was more effective
Null Hypothesis - The experimental homepage was not effective
Alternate Hypothesis - The experimental homepage was effective
'''


# Calculate CTR per group
ctr = user_actions.groupby("group")["clicked"].mean()
counts = user_actions.groupby("group")["clicked"].agg(["sum","count"])
print("CTR per group:\n", ctr)
print("Counts:\n", counts)

CTR per group:
 group
control       0.279712
experiment    0.309746
Name: clicked, dtype: float64
Counts:
             sum  count
group                 
control     932   3332
experiment  928   2996


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not.

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group.
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate.
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group.

In [ ]:
#Your code here
# For each user, did they click?
user_actions = df.groupby(["id", "group"])["action"].apply(lambda x: "click" in x.values).reset_index()

# Convert to numeric (1 = clicked, 0 = did not click)
user_actions["clicked"] = user_actions["action"].astype(int)

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [ ]:
#Your code here
std_dev = user_actions['clicked'].std()
std_dev

np.float64(0.4555970484323387)

### Step 3:
Finally, calculate a p-value using the normal distribution based on this z-score.

In [ ]:
#Your code here
from statsmodels.stats.proportion import proportions_ztest

# Extract successes (clicks) and totals
successes = counts["sum"].values
nobs = counts["count"].values

# Run two-proportion z-test
stat, pval = proportions_ztest(successes, nobs)

print("z-statistic:", stat, "p-value:", pval)

z-statistic: -2.618563885349469 p-value: 0.008830075576595804


### Analysis:

Does this result roughly match that of the previous statistical test?

# **The p-value is less than 0.05 so we reject the null hypothesis and accept that the experimental homepage did improve the number of clicks**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.